In [2]:
#This program is a game! You will guess what side a 6 sided dice will land on
#I was originaly just going to have you roll the dice and have it count but then figured this would be cooler and fun. It was so much harder and i had to use
#other sources and friends to help me with some of the functions but eventually it ended up really cool!
#Hope you enjoy!



import tkinter as tk 
from tkinter import messagebox #A module that has a set of function to make actions in the tkinter GUI!
import random
import sqlite3 #For the database

class dicerollerapp:
    def __init__(self, master):
        self.master = master
        self.master.title("Dice Roller")

        # Connecting to my SQLite3 database where i can store the data!
        self.conn = sqlite3.connect('dicerolls.db')
        self.create_table_if_not_exists() #if i dont already have one

        self.label = tk.Label(master, text="Guess the roll (1-6) and click ROLL!!!")
        self.label.pack() #Prompting you to guess what it will be, the GAME!

        self.guess_entry = tk.Entry(master)
        self.guess_entry.pack() #User entry

        self.roll_value_label = tk.Label(master, text="")
        self.roll_value_label.pack()

        self.roll_button = tk.Button(master, text="ROLL!!!", command=self.roll_dice, bg="blue", fg="white", cursor="hand2")
        self.roll_button.pack() #The button to roll with other cool features from the activity!

        self.result_label = tk.Label(master, text="")
        self.result_label.pack()

    def create_table_if_not_exists(self): #Storing the roll data in a table
        cursor = self.conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS rolls (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        roll_value INTEGER, 
                        user_guess INTEGER,
                        roll_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
        self.conn.commit() 

    def roll_dice(self):
        user_guess = self.guess_entry.get()
        if not user_guess.isdigit() or int(user_guess) not in range(1, 7):
            messagebox.showerror("Invalid Guess", "Please enter a number between 1 and 6 as your guess.")
            return #error checking(THIS was a pain to figure out and display in the box haha)

        user_guess = int(user_guess)
        roll_result = random.randint(1, 6)  # Roll a six-sided dice
        self.roll_value_label.config(text=f"Roll Value: {roll_result}")
        self.result_label.config(text=f"Rolled: {roll_result}") #I was excited to put in f strings, i have never used them in a project till this class!

        self.save_roll_to_db(roll_result, user_guess)

        if user_guess == roll_result:
            messagebox.showinfo("Congratulations!", f"You guessed right! The roll number was {roll_result}. Data saved.") #When they guess right
            self.master.destroy()  # Closing the GUI window when you guess right!
        else:
            messagebox.showinfo("Wrong Guess", f"WRONG, the roll was {roll_result}. Keep guessing!")#When you guess wrong

    def save_roll_to_db(self, roll_result, user_guess):
        cursor = self.conn.cursor()
        cursor.execute("INSERT INTO rolls (roll_value, user_guess) VALUES (?, ?)", (roll_result, user_guess))
        self.conn.commit()#Saving the data of the rolls, this also took a lot of time to figure out

    def get_roll_stats_from_db(self):
        cursor = self.conn.cursor()
        cursor.execute("SELECT roll_value, user_guess, COUNT(*) FROM rolls GROUP BY roll_value, user_guess")
        return cursor.fetchall() #Select query for roll data, not going to lie taking the database managment class while taking this class helped me with this part too!


def main():
    root = tk.Tk()
    app = dicerollerapp(root)
    root.mainloop()

if __name__ == "__main__":
    main()